# LangChain ✕ MLRun Integration

`langchain_mlrun` is a hub module that implements LangChain integration with MLRun. Using the module allows MLRun to orchestrate LangChain and LangGraph code, enabling tracing and monitoring batch workflows and realtime deployments.
___

## Main Components

This is a short brief of the components available to import from the `langchain_mlrun` module. For full docs, see the documentation page.

### Settings

The module uses Pydantic settings classes that can be configured programmatically or via environment variables. The main class is `MLRunTracerSettings`. It contains two sub-settings:
* `MLRunTracerClientSettings` - Connection settings (stream path, container, endpoint info). Env prefix: `"MLRUN_TRACER_CLIENT_"`
* `MLRunTracerMonitorSettings` - Controls what/how runs are captured (filters, labels, debug mode). Env prefix: `"MLRUN_TRACER_MONITOR_"`

For more information about each setting, see the class docstrings.

#### Example - via code configuration

```python
from langchain_mlrun import MLRunTracerSettings, MLRunTracerClientSettings, MLRunTracerMonitorSettings

settings = MLRunTracerSettings(
    client=MLRunTracerClientSettings(
        stream_path="my-project/model-endpoints/stream-v1",
        container="projects",
        model_endpoint_name="my_endpoint",
        model_endpoint_uid="abc123",
        serving_function="my_function",
    ),
    monitor=MLRunTracerMonitorSettings(
        label="production",
        root_run_only=True,  # Only monitor root runs, not child runs
        tags_filter=["important"],  # Only monitor runs with this tag
    ),
)
```

#### Example - environment variable configuration

```bash
export MLRUN_TRACER_CLIENT_STREAM_PATH="my-project/model-endpoints/stream-v1"
export MLRUN_TRACER_CLIENT_CONTAINER="projects"
export MLRUN_TRACER_MONITOR_LABEL="production"
export MLRUN_TRACER_MONITOR_ROOT_RUN_ONLY="true"
```

### MLRun Tracer

`MLRunTracer` is a LangChain-compatible tracer that converts LangChain `Run` objects into MLRun monitoring events and publishes them to a V3IO stream. 

Key points:
* **No inheritance required** - use it directly without subclassing.
* **Fully customizable via settings** - control filtering, summarization, and output format.
* **Custom summarizer support** - pass your own `run_summarizer_function` via settings to customize how runs are converted to events.

### Monitoring Setup Utility Function

`setup_langchain_monitoring()` is a utility function that creates the necessary MLRun infrastructure for LangChain monitoring. This is a **temporary workaround** until custom endpoint creation support is added to MLRun.

The function returns a dictionary of environment variables to configure auto-tracing. See how to use it in the tutorial section below.

### LangChain Monitoring Application

`LangChainMonitoringApp` is a base class (inheriting from MLRun's `ModelMonitoringApplicationBase`) for building monitoring applications that process events from the MLRun Tracer.

It offers several built-in helper methods and metrics for analyzing LangChain runs:

* Helper methods:
  * `get_structured_runs()` - Parse raw monitoring samples into structured run dictionaries with filtering options
  * `iterate_structured_runs()` - Iterate over all runs including nested child runs
* Metric methods:
  * `calculate_average_latency()` - Average latency across root runs
  * `calculate_success_rate()` - Percentage of runs without errors
  * `count_token_usage()` - Total input/output tokens from LLM runs
  * `count_run_names()` - Count occurrences of each run name

The base app can be used as-is, but it is recommended to extend it with your own custom monitoring logic.
___

## How to Apply MLRun?

### Auto Tracing

Auto tracing automatically instruments all LangChain code by setting the `MLRUN_MONITORING_ENABLED` environment variable and importing the module:

```python
import os
os.environ["MLRUN_MONITORING_ENABLED"] = "1"
# Set other MLRUN_TRACER_* environment variables as needed...

# Import the module BEFORE any LangChain code
langchain_mlrun = mlrun.import_module("hub://langchain_mlrun")

# All LangChain/LangGraph code below will be automatically traced
chain.invoke(...)
```

### Manual Tracing

For more control, use the `mlrun_monitoring()` context manager to trace specific code blocks:

```python
from langchain_mlrun import mlrun_monitoring, MLRunTracerSettings

# Optional: customize settings
settings = MLRunTracerSettings(...)

with mlrun_monitoring(settings=settings) as tracer:
    # Only LangChain code within this block will be traced
    result = chain.invoke({"topic": "MLRun"})
```
___

## Tutorial

In this tutorial we'll show how to orchestrate LangChain based code with MLRun using the `langchain_mlrun` hub module.

### Prerequisites

Install MLRun and the `langchain_mlrun` requirements.

In [ ]:
!pip install mlrun langchain pydantic-settings

### Create Project

We'll first create an MLRun project

In [2]:
import os
import time
import datetime
import mlrun

project = mlrun.get_or_create_project("langchain-mlrun-tutorial")

> 2026-01-08 14:48:52,259 [info] Project loaded successfully: {"project_name":"langchain-mlrun-7"}


### Enable Monitoring

To use MLRun's monitoring feature in our project we first need to set up the monitoring infrastructure. If you use MLRun CE, you'll need to create a Kafka stream, if you use MLRun enterprise, you can use V3IO.

In [3]:
# TODO: Add here MLRun CE handler with Kafka, currently the tutorial is only with V3IO.
from mlrun.datastore import DatastoreProfileV3io

# Create a V3IO data store:
v3io_ds = DatastoreProfileV3io(name="v3io-ds",v3io_access_key=os.environ["V3IO_ACCESS_KEY"])
project.register_datastore_profile(profile=v3io_ds)

# Set the monitoring credentials:
project.set_model_monitoring_credentials(
    stream_profile_name=v3io_ds.name,
    tsdb_profile_name=v3io_ds.name
)

# Enable monitoring for our project:
project.enable_model_monitoring(
    base_period=1,
    wait_for_deployment=True,
)

### Import `langchain_mlrun`

Now we'll import `langchain_mlrun` from the hub.

In [ ]:
# Import the module from the hub:
langchain_mlrun = mlrun.import_module("hub://langchain_mlrun")

# Import the utility function and monitoring application from the module:
setup_langchain_monitoring = langchain_mlrun.setup_langchain_monitoring
LangChainMonitoringApp = langchain_mlrun.LangChainMonitoringApp

### Create Monitorable Endpoint

Endpoints are the entities being monitored by MLRun. As such we'll use the `setup_langchain_monitoring` utility function to create the model monitoring endpoint. By default, our endpoint name will be `"langchain_mlrun_endpoint"` but feel free to change it by using the required arguments.

In [5]:
env_vars = setup_langchain_monitoring()

Creating LangChain model endpoint

  [✓] Loading Project......................... Done (0.00s)
  [✓] Creating Model.......................... Done (0.02s)               
  [✓] Creating Function....................... Done (0.02s)                  
  [✓] Creating Model Endpoint................. Done (0.02s)                  

✨ Done! LangChain monitoring model endpoint created successfully.
You can now set the following environment variables to enable MLRun tracing in your LangChain code:

{
    "MLRUN_MONITORING_ENABLED": "1",
    "MLRUN_TRACER_CLIENT_PROJECT": "langchain-mlrun-7",
    "MLRUN_TRACER_CLIENT_STREAM_PATH": "langchain-mlrun-7/model-endpoints/stream-v1",
    "MLRUN_TRACER_CLIENT_CONTAINER": "projects",
    "MLRUN_TRACER_CLIENT_MODEL_ENDPOINT_NAME": "langchain_mlrun_endpoint",
    "MLRUN_TRACER_CLIENT_MODEL_ENDPOINT_UID": "bb81af2058c14e7cbf58455aed3d69fc",
    "MLRUN_TRACER_CLIENT_SERVING_FUNCTION": "langchain_mlrun_function"
}

To customize the monitoring behavior, you can

### Setup Environment Variables for Auto Tracing

We'll use the environment variables returned from `setup_langchain_monitoring` to setup the environment for auto-tracing. Read the printed outputs for more information.

In [ ]:
os.environ.update(env_vars)

### Run `langchain` or `langgraph` Code

Here we have 3 functions, each using different method utilizing LLMs with `langchain` and `langgraph`:
* `run_simple_chain` - Using `langchain`'s chains.
* `run_simple_agent` - Using `langchain`'s `create_agent` function and `tool`s.
* `run_langgraph_graph` - Using pure `langgraph`.

> **Notice**: You don't need to set OpenAI API credentials, there is a mock `ChatModel` that will replace it if the credentials are not set in the environment. If you wish to use OpenAI models, make sure you `pip install langchain_openai` and set the `OPENAI_API_KEY` environment variable before continue to the next cell.

Because the auto-tracing environment is set, any run will be automatically traced and monitored!

Feel free to adjust the code as you like.

> **Remember**: To enable auto-tracing you do need to set the environment variables and import the `langchain_mlrun` module before any LangChain code. For batch jobs and realtime functions, make sure you set env vars in the MLRun function and add the import line `langchain_mlrun = mlrun.import_module("hub://langchain_mlrun")` at the top of your code.

In [7]:
import os
from typing import Literal, TypedDict, Annotated, Sequence, Any, Callable
from operator import add

from langchain_core.language_models import LanguageModelInput
from langchain_core.runnables import Runnable, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.language_models.fake_chat_models import FakeListChatModel, GenericFakeChatModel
from langchain.agents import create_agent
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.tools import tool, BaseTool

from langgraph.graph import StateGraph, START, END
from langchain_core.messages import BaseMessage


def _check_openai_credentials() -> bool:
    """
    Check if OpenAI API key is set in environment variables.

    :return: True if OPENAI_API_KEY is set, False otherwise.
    """
    return "OPENAI_API_KEY" in os.environ


# Import ChatOpenAI only if OpenAI credentials are available (meaning `langchain-openai` must be installed).
if _check_openai_credentials():
    from langchain_openai import ChatOpenAI

    
class _ToolEnabledFakeModel(GenericFakeChatModel):
    """
    A fake chat model that supports tool binding for running agent tracing tests.
    """

    def bind_tools(
        self,
        tools: Sequence[
            dict[str, Any] | type | Callable | BaseTool  # noqa: UP006
        ],
        *,
        tool_choice: str | None = None,
        **kwargs: Any,
    ) -> Runnable[LanguageModelInput, AIMessage]:
        return self


#: Tag value for testing tag filtering.
_dummy_tag = "dummy_tag"


def run_simple_chain() -> str:
    """
    Run a simple LangChain chain that gets a fact about a topic.
    """
    # Build a simple chain: prompt -> llm -> str output parser
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        tags=[_dummy_tag]
    ) if _check_openai_credentials() else (
        FakeListChatModel(
            responses=[
                "MLRun is an open-source orchestrator for machine learning pipelines."
            ],
            tags=[_dummy_tag]
        )
    )
    prompt = ChatPromptTemplate.from_template("Tell me a short fact about {topic}")
    chain = prompt | llm | StrOutputParser()

    # Run the chain:
    response = chain.invoke({"topic": "MLRun"})
    return response


def run_simple_agent():
    """
    Run a simple LangChain agent that uses two tools to get weather and stock price.
    """
    # Define the tools:
    @tool
    def get_weather(city: str) -> str:
        """Get the current weather for a specific city."""
        return f"The weather in {city} is 22°C and sunny."

    @tool
    def get_stock_price(symbol: str) -> str:
        """Get the current stock price for a symbol."""
        return f"The stock price for {symbol} is $150.25."

    # Define the model:
    model = ChatOpenAI(
        model="gpt-4o-mini",
        tags=[_dummy_tag]
    ) if _check_openai_credentials() else (
        _ToolEnabledFakeModel(
            messages=iter(
                [
                    AIMessage(
                        content="",
                        tool_calls=[
                            {"name": "get_weather", "args": {"city": "London"}, "id": "call_abc123"},
                            {"name": "get_stock_price", "args": {"symbol": "AAPL"}, "id": "call_def456"}
                        ]
                    ),
                    AIMessage(content="The weather in London is 22°C and AAPL is trading at $150.25.")
                ]
            ),
            tags=[_dummy_tag]
        )
    )

    # Create the agent:
    agent = create_agent(
        model=model,
        tools=[get_weather, get_stock_price],
        system_prompt="You are a helpful assistant with access to tools."
    )

    # Run the agent:
    return agent.invoke({"messages": ["What is the weather in London and the stock price of AAPL?"]})


def run_langgraph_graph():
    """
    Run a LangGraph agent that uses reflection to correct its answer.
    """
    # Define the graph state:
    class AgentState(TypedDict):
        messages: Annotated[list[BaseMessage], add]
        attempts: int

    # Define the model:
    model = ChatOpenAI(model="gpt-4o-mini") if _check_openai_credentials() else (
        _ToolEnabledFakeModel(
            messages=iter(
                [
                    AIMessage(content="There are 2 'r's in Strawberry."),  # Mocking the failure
                    AIMessage(content="I stand corrected. S-t-r-a-w-b-e-r-r-y. There are 3 'r's."),  # Mocking the fix
                ]
            )
        )
    )

    # Define the graph nodes and router:
    def call_model(state: AgentState):
        response = model.invoke(state["messages"])
        return {"messages": [response], "attempts": state["attempts"] + 1}

    def reflect_node(state: AgentState):
        prompt = "Wait, count the 'r's again slowly, letter by letter. Are you sure?"
        return {"messages": [HumanMessage(content=prompt)]}

    def router(state: AgentState) -> Literal["reflect", END]:
        # Make sure there are 2 attempts at least for an answer:
        if state["attempts"] == 1:
            return "reflect"
        return END

    # Build the graph:
    builder = StateGraph(AgentState)
    builder.add_node("model", call_model)
    tagged_reflect_node = RunnableLambda(reflect_node).with_config(tags=[_dummy_tag])
    builder.add_node("reflect", tagged_reflect_node)
    builder.add_edge(START, "model")
    builder.add_conditional_edges("model", router)
    builder.add_edge("reflect", "model")
    graph = builder.compile()

    # Run the graph:
    return graph.invoke({"messages": [HumanMessage(content="How many 'r's in Strawberry?")], "attempts": 0})

Let's create some traffic, we'll run whatever function you want in a loop to get some events. We take timestamps in order to use them later to run the monitoring application on the data we'll send.

In [11]:
# Run LangChain code and now it should be tracked and monitored in MLRun:
start_timestamp = datetime.datetime.now() - datetime.timedelta(minutes=1)
for i in range(20):
    run_simple_agent()
end_timestamp = datetime.datetime.now() + datetime.timedelta(minutes=5)

> **Note**: Please wait a minute or two until the events are processed.

In [ ]:
time.sleep(60)

### Test the LangChain Monitoring Application

To test a monitoring application, we use the `evaluate` class method. We'll run an evaluation on the data we just sent. It is a small local job and should run fast.

Keep an eye for the returned metrics from the monitoring application.

In [9]:
LangChainMonitoringApp.evaluate(
    func_name="langchain_monitoring_app_test",
    func_path="langchain_mlrun.py",
    run_local=True,
    endpoints=[env_vars["MLRUN_TRACER_CLIENT_MODEL_ENDPOINT_NAME"]],
    start=start_timestamp.isoformat(),
    end=end_timestamp.isoformat(),
)

> 2026-01-08 14:49:22,970 [info] Changing function name - adding `"-batch"` suffix: {"func_name":"testi-batch"}
> 2026-01-08 14:49:23,143 [info] Storing function: {"db":"http://mlrun-api:8080","name":"testi-batch--handler","uid":"43b34f848b6049c0949f04adc1090f10"}


project,uid,iter,start,end,state,kind,name,labels,inputs,parameters,results
langchain-mlrun-7,...c1090f10,0,Jan 08 14:49:23,NaT,completed,run,testi-batch--handler,v3io_user=guylkind=localowner=guylhost=jupyter-guyl-66647f988c-4kjd9,,endpoints=['langchain_mlrun_endpoint']start=2026-01-08T10:19:47.452879end=2026-01-08T10:26:28.861851base_period=Nonewrite_output=Falseexisting_data_handling=fail_on_overlapstream_profile=None,"langchain_mlrun_endpoint-bb81af2058c14e7cbf58455aed3d69fc_2026-01-08T10:19:47.452879+00:00_2026-01-08T10:26:28.861851+00:00=[{metric_name: 'average_latency', metric_value: 1949.3444}, {metric_name: 'success_rate', metric_value: 1.0}, {metric_name: 'total_input_tokens', metric_value: 5480.0}, {metric_name: 'total_output_tokens', metric_value: 1404.0}, {metric_name: 'combined_total_tokens', metric_value: 6884.0}, {metric_name: 'run_name_counts_ChatOpenAI', metric_value: 40.0}, {metric_name: 'run_name_counts_model', metric_value: 40.0}, {metric_name: 'run_name_counts_get_weather', metric_value: 20.0}, {metric_name: 'run_name_counts_tools', metric_value: 40.0}, {metric_name: 'run_name_counts_get_stock_price', metric_value: 20.0}, {metric_name: 'run_name_counts_LangGraph', metric_value: 20.0}]"


> 2026-01-08 14:49:23,944 [info] Run execution finished: {"name":"testi-batch--handler","status":"completed"}


### Deploy the Monitoring Application

All that's left to do now is to deploy our monitoring application!

In [10]:
# Deploy the monitoring app:
LangChainMonitoringApp.deploy(
    func_name="langchain_monitoring_app",
    func_path="langchain_mlrun.py",
    image="mlrun/mlrun",
    requirements=[
        "langchain",
        "pydantic-settings",
    ],
)

> 2026-01-08 17:06:50,801 [info] Starting remote function deploy
2026-01-08 17:06:51  (info) Deploying function
2026-01-08 17:06:51  (info) Building
2026-01-08 17:06:52  (info) Staging files and preparing base images
2026-01-08 17:06:52  (warn) Using user provided base image, runtime interpreter version is provided by the base image
2026-01-08 17:06:52  (info) Building processor image
2026-01-08 17:08:52  (info) Build complete
2026-01-08 17:09:06  (info) Function deploy complete
> 2026-01-08 17:09:13,972 [info] Model endpoint creation task completed with state succeeded
> 2026-01-08 17:09:13,973 [info] Successfully deployed function: {"external_invocation_urls":[],"internal_invocation_urls":["nuclio-langchain-mlrun-7-langchain-monitoring-app.default-tenant.svc.cluster.local:8080"]}


Once it is deployed, you can run events again and see the monitoring application in MLRun UI in action:

![mlrun ui example](./notebook_images/mlrun_ui.png)